# Обработка pdf-файла

In [1]:
!pip install sys
!pip install fitz

Looking in indexes: https://artifactory.csp.aimdt.com/artifactory/api/pypi/pub-pypi/simple
ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys
Looking in indexes: https://artifactory.csp.aimdt.com/artifactory/api/pypi/pub-pypi/simple


### Считывание файла

In [2]:

import os
import sys
import fitz

def column_boxes(page, footer_margin=50, header_margin=50, no_image_text=True):
    """Determine bboxes which wrap a column."""
    paths = page.get_drawings()
    bboxes = []

    # path rectangles
    path_rects = []

    # image bboxes
    img_bboxes = []

    # bboxes of non-horizontal text
    # avoid when expanding horizontal text boxes
    vert_bboxes = []

    # compute relevant page area
    clip = +page.rect
    clip.y1 -= footer_margin  # Remove footer area
    clip.y0 += header_margin  # Remove header area

    def can_extend(temp, bb, bboxlist):
        """Determines whether rectangle 'temp' can be extended by 'bb'
        without intersecting any of the rectangles contained in 'bboxlist'.

        Items of bboxlist may be None if they have been removed.

        Returns:
            True if 'temp' has no intersections with items of 'bboxlist'.
        """
        for b in bboxlist:
            if not intersects_bboxes(temp, vert_bboxes) and (
                b == None or b == bb or (temp & b).is_empty
            ):
                continue
            return False

        return True

    def in_bbox(bb, bboxes):
        """Return 1-based number if a bbox contains bb, else return 0."""
        for i, bbox in enumerate(bboxes):
            if bb in bbox:
                return i + 1
        return 0

    def intersects_bboxes(bb, bboxes):
        """Return True if a bbox intersects bb, else return False."""
        for bbox in bboxes:
            if not (bb & bbox).is_empty:
                return True
        return False

    def extend_right(bboxes, width, path_bboxes, vert_bboxes, img_bboxes):
        """Extend a bbox to the right page border.

        Whenever there is no text to the right of a bbox, enlarge it up
        to the right page border.

        Args:
            bboxes: (list[IRect]) bboxes to check
            width: (int) page width
            path_bboxes: (list[IRect]) bboxes with a background color
            vert_bboxes: (list[IRect]) bboxes with vertical text
            img_bboxes: (list[IRect]) bboxes of images
        Returns:
            Potentially modified bboxes.
        """
        for i, bb in enumerate(bboxes):
            # do not extend text with background color
            if in_bbox(bb, path_bboxes):
                continue

            # do not extend text in images
            if in_bbox(bb, img_bboxes):
                continue

            # temp extends bb to the right page border
            temp = +bb
            temp.x1 = width

            # do not cut through colored background or images
            if intersects_bboxes(temp, path_bboxes + vert_bboxes + img_bboxes):
                continue

            # also, do not intersect other text bboxes
            check = can_extend(temp, bb, bboxes)
            if check:
                bboxes[i] = temp  # replace with enlarged bbox

        return [b for b in bboxes if b != None]

    def clean_nblocks(nblocks):
        """Do some elementary cleaning."""

        # 1. remove any duplicate blocks.
        blen = len(nblocks)
        if blen < 2:
            return nblocks
        start = blen - 1
        for i in range(start, -1, -1):
            bb1 = nblocks[i]
            bb0 = nblocks[i - 1]
            if bb0 == bb1:
                del nblocks[i]

        # 2. repair sequence in special cases:
        # consecutive bboxes with almost same bottom value are sorted ascending
        # by x-coordinate.
        y1 = nblocks[0].y1  # first bottom coordinate
        i0 = 0  # its index
        i1 = -1  # index of last bbox with same bottom

        # Iterate over bboxes, identifying segments with approx. same bottom value.
        # Replace every segment by its sorted version.
        for i in range(1, len(nblocks)):
            b1 = nblocks[i]
            if abs(b1.y1 - y1) > 10:  # different bottom
                if i1 > i0:  # segment length > 1? Sort it!
                    nblocks[i0 : i1 + 1] = sorted(
                        nblocks[i0 : i1 + 1], key=lambda b: b.x0
                    )
                y1 = b1.y1  # store new bottom value
                i0 = i  # store its start index
            i1 = i  # store current index
        if i1 > i0:  # segment waiting to be sorted
            nblocks[i0 : i1 + 1] = sorted(nblocks[i0 : i1 + 1], key=lambda b: b.x0)
        return nblocks

    # extract vector graphics
    for p in paths:
        path_rects.append(p["rect"].irect)
    path_bboxes = path_rects

    # sort path bboxes by ascending top, then left coordinates
    path_bboxes.sort(key=lambda b: (b.y0, b.x0))

    # bboxes of images on page, no need to sort them
    for item in page.get_images():
        img_bboxes.extend(page.get_image_rects(item[0]))

    # blocks of text on page
    blocks = page.get_text(
        "dict",
        flags=fitz.TEXTFLAGS_TEXT,
        clip=clip,
    )["blocks"]

    # Make block rectangles, ignoring non-horizontal text
    for b in blocks:
        bbox = fitz.IRect(b["bbox"])  # bbox of the block

        # ignore text written upon images
        if no_image_text and in_bbox(bbox, img_bboxes):
            continue

        # confirm first line to be horizontal
        line0 = b["lines"][0]  # get first line
        if line0["dir"] != (1, 0):  # only accept horizontal text
            vert_bboxes.append(bbox)
            continue

        srect = fitz.EMPTY_IRECT()
        for line in b["lines"]:
            lbbox = fitz.IRect(line["bbox"])
            text = "".join([s["text"].strip() for s in line["spans"]])
            if len(text) > 1:
                srect |= lbbox
        bbox = +srect

        if not bbox.is_empty:
            bboxes.append(bbox)

    # Sort text bboxes by ascending background, top, then left coordinates
    bboxes.sort(key=lambda k: (in_bbox(k, path_bboxes), k.x0, k.y0))

    # Extend bboxes to the right where possible
    bboxes = extend_right(
        bboxes, int(page.rect.width), path_bboxes, vert_bboxes, img_bboxes
    )

    # immediately return of no text found
    if bboxes == []:
        return []

    # --------------------------------------------------------------------
    # Join bboxes to establish some column structure
    # --------------------------------------------------------------------
    # the final block bboxes on page
    nblocks = [bboxes[0]]  # pre-fill with first bbox
    bboxes = bboxes[1:]  # remaining old bboxes

    for i, bb in enumerate(bboxes):  # iterate old bboxes
        check = False  # indicates unwanted joins

        # check if bb can extend one of the new blocks
        for j in range(len(nblocks)):
            nbb = nblocks[j]  # a new block

            # never join across columns
            if bb == None or nbb.x1 < bb.x0 or bb.x1 < nbb.x0:
                continue

            # never join across different background colors
            if in_bbox(nbb, path_bboxes) != in_bbox(bb, path_bboxes):
                continue

            temp = bb | nbb  # temporary extension of new block
            check = can_extend(temp, nbb, nblocks)
            if check == True:
                break

        if not check:  # bb cannot be used to extend any of the new bboxes
            nblocks.append(bb)  # so add it to the list
            j = len(nblocks) - 1  # index of it
            temp = nblocks[j]  # new bbox added

        # check if some remaining bbox is contained in temp
        check = can_extend(temp, bb, bboxes)
        if check == False:
            nblocks.append(bb)
        else:
            nblocks[j] = temp
        bboxes[i] = None

    # do some elementary cleaning
    nblocks = clean_nblocks(nblocks)

    # return identified text bboxes
    return nblocks


In [3]:
import fitz  # это pymupdf
import re

def extract_paragraphs_from_pdf(pdf_path):
    """
    Извлекает текст из PDF файла и возвращает список абзацев.

    :param pdf_path: Путь к PDF файлу.
    :return: Список абзацев, где каждый абзац представлен как строка.
    """
    # Открытие PDF документа
    doc = fitz.open(pdf_path)

    # Переменная для хранения всего извлеченного текста
    all_extracted_text = ""

    # Обработка каждой страницы
    for page in doc:
        # Получение границ текстовых блоков
        bboxes = column_boxes(page, footer_margin=50, no_image_text=True)

        # Извлечение текста из каждого блока и добавление в строку
        for rect in bboxes:
            # Извлекаем текст для текущего прямоугольника
            extracted_text = page.get_text(clip=rect, sort=True)
            # Добавляем текст в общую строку
            all_extracted_text += extracted_text
            
    return all_extracted_text

# Пример использования функции
pdf_path = "Coal_Trader_16_Sep_2024.pdf"
paragraphs = extract_paragraphs_from_pdf(pdf_path)
print(paragraphs)

Coal Trader
 Monday, September 16, 2024

US OTC coals open week with unchanged finishes      Platts daily physical OTC assessments, Sep 16   NYMEX natural gas contract adds 6.8 cents

  US over-the-counter coals opened the week on Sept. 16 with a
slate of stable assessments, holding with average temperatures
expected nationwide in this shoulder season.
  S&P Global PointLogic data points to temperatures within 2
degrees Fahrenheit of seasonal norms through the end of the
month. The Northeast forecast follows that expectation, while
Midcontinent temperatures could rise to 6 degrees above normal
in the next week. Southeast temperatures are expected to stay at
or below seasonal averages through September.
                                                           (continued on page 9)Inside this Issue
Asian met coal market sees fresh trade at lower level; sellers await China’s return.. 2
Energy Transfer NGLs line catches fire near Houston, forcing evacuations........... 2
New York gets f

### Обработка от исходного текста до списка абзацев

In [4]:
def process_paragraphs(paragraphs):
    lines = paragraphs.splitlines()  # Разделяем текст на строки
    result = []  # Список для хранения обработанных строк

    for i in range(len(lines)):
        line = lines[i]

        # Условие 1: Если строка начинается с пробела, добавляем её без изменений
        if line.startswith(' '):
            result.append(line)
        else:
            # Условие 2: Если строка не начинается с пробела и не содержит табуляции
            if '  ' not in line:
                # Проверяем, что предыдущая строка не пустая
                if i > 0 and lines[i - 1].strip():  # Если предыдущая строка не пустая
                    # Объединяем текущую строку с предыдущей, убирая переход на новую строку
                    result[-1] += ' ' + line.strip()  # Добавляем текущую строку без перехода
                    continue  # Переходим к следующей строке
            result.append(line)  # Добавляем текущую строку

    return '\n'.join(result)  # Объединяем строки обратно в текст



processed_text = process_paragraphs(paragraphs)
print(processed_text)

Coal Trader
 Monday, September 16, 2024 
US OTC coals open week with unchanged finishes      Platts daily physical OTC assessments, Sep 16   NYMEX natural gas contract adds 6.8 cents 
  US over-the-counter coals opened the week on Sept. 16 with a slate of stable assessments, holding with average temperatures expected nationwide in this shoulder season.
  S&P Global PointLogic data points to temperatures within 2 degrees Fahrenheit of seasonal norms through the end of the month. The Northeast forecast follows that expectation, while Midcontinent temperatures could rise to 6 degrees above normal in the next week. Southeast temperatures are expected to stay at or below seasonal averages through September.
                                                           (continued on page 9)Inside this Issue Asian met coal market sees fresh trade at lower level; sellers await China’s return.. 2 Energy Transfer NGLs line catches fire near Houston, forcing evacuations........... 2 New York gets f

In [5]:
def process_and_merge_tabbed_lines(processed_text):
    result = []
    buffer = []  # Временный буфер для строк с табуляцией

    # Удаляем пробелы в начале каждой строки
    lines = [line.lstrip() for line in processed_text.splitlines()]

    for line in lines:
        if '  ' in line:
            # Если строка содержит табуляцию, добавляем её в буфер
            buffer.append(line)
        else:
            if buffer:
                # Если в буфере есть строки с табуляцией, объединяем их и добавляем в результат
                result.append("\n".join(buffer))
                buffer = []  # Очищаем буфер после добавления

            # Добавляем обычную строку (без табуляции) в результат
            result.append(line)

    # Добавляем оставшиеся строки с табуляцией, если они есть в конце текста
    if buffer:
        result.append("\n".join(buffer))

    return result

lines_list = process_and_merge_tabbed_lines(processed_text)
lines_list

['Coal Trader',
 'Monday, September 16, 2024 ',
 'US OTC coals open week with unchanged finishes      Platts daily physical OTC assessments, Sep 16\x84 \u2003 NYMEX natural gas contract adds 6.8 cents ',
 'US over-the-counter coals opened the week on Sept. 16 with a slate of stable assessments, holding with average temperatures expected nationwide in this shoulder season.',
 'S&P Global PointLogic data points to temperatures within 2 degrees Fahrenheit of seasonal norms through the end of the month. The Northeast forecast follows that expectation, while Midcontinent temperatures could rise to 6 degrees above normal in the next week. Southeast temperatures are expected to stay at or below seasonal averages through September.',
 '(continued on page 9)Inside this Issue Asian met coal market sees fresh trade at lower level; sellers await China’s return.. 2 Energy Transfer NGLs line catches fire near Houston, forcing evacuations........... 2 New York gets funding to expand EV fast charging,

In [6]:
def process_list_elements(lines_list):
    result = []

    for i, line in enumerate(lines_list):
        # Пропускаем пустые элементы
        if not line.strip():
            continue

        # Если это не первый элемент и текущий элемент начинается с маленькой буквы,
        # а в предыдущем элементе нет двойного пробела, объединяем с предыдущим
        if result and line[0].islower() and '  ' not in result[-1]:
            result[-1] = result[-1].rstrip() + ' ' + line.lstrip()
        else:
            result.append(line)

    return result

# Применяем новый этап обработки
processed_list = process_list_elements(lines_list)
processed_list

['Coal Trader',
 'Monday, September 16, 2024 ',
 'US OTC coals open week with unchanged finishes      Platts daily physical OTC assessments, Sep 16\x84 \u2003 NYMEX natural gas contract adds 6.8 cents ',
 'US over-the-counter coals opened the week on Sept. 16 with a slate of stable assessments, holding with average temperatures expected nationwide in this shoulder season.',
 'S&P Global PointLogic data points to temperatures within 2 degrees Fahrenheit of seasonal norms through the end of the month. The Northeast forecast follows that expectation, while Midcontinent temperatures could rise to 6 degrees above normal in the next week. Southeast temperatures are expected to stay at or below seasonal averages through September.',
 '(continued on page 9)Inside this Issue Asian met coal market sees fresh trade at lower level; sellers await China’s return.. 2 Energy Transfer NGLs line catches fire near Houston, forcing evacuations........... 2 New York gets funding to expand EV fast charging,

### Лемматизация текста с использованием NLTK

In [10]:
import nltk
import os

# Указываем путь к папке nltk_data
nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')
nltk.data.path.append(nltk_data_path)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [11]:
# Лемматизатор
lemmatizer = WordNetLemmatizer()

# Стоп-слова для английского (можно добавить стоп-слова для других языков)
stop_words = set(stopwords.words('english'))

In [12]:
# Функция для очистки текста: удаление стоп-слов и лемматизация
def process_text(text):
    # Токенизация
    tokens = word_tokenize(text)

    # Удаление стоп-слов и лемматизация
    cleaned_tokens = [
        lemmatizer.lemmatize(word.lower()) for word in tokens
        if word.isalnum() and word.lower() not in stop_words  # Убираем стоп-слова и небуквенные токены
    ]

    return cleaned_tokens

In [13]:
# Применение ко всему списку
result = [process_text(text) for text in processed_list]

# Вывод результата
for i, cleaned in enumerate(result):
    print(f"Оригинал: {processed_list[i]}")
    print(f"Очищенные и лемматизированные токены: {cleaned}")

Оригинал: Coal Trader
Очищенные и лемматизированные токены: ['coal', 'trader']
Оригинал: Monday, September 16, 2024 
Очищенные и лемматизированные токены: ['monday', 'september', '16', '2024']
Оригинал: US OTC coals open week with unchanged finishes      Platts daily physical OTC assessments, Sep 16   NYMEX natural gas contract adds 6.8 cents 
Очищенные и лемматизированные токены: ['u', 'otc', 'coal', 'open', 'week', 'unchanged', 'finish', 'platts', 'daily', 'physical', 'otc', 'assessment', 'sep', 'nymex', 'natural', 'gas', 'contract', 'add', 'cent']
Оригинал: US over-the-counter coals opened the week on Sept. 16 with a slate of stable assessments, holding with average temperatures expected nationwide in this shoulder season.
Очищенные и лемматизированные токены: ['u', 'coal', 'opened', 'week', '16', 'slate', 'stable', 'assessment', 'holding', 'average', 'temperature', 'expected', 'nationwide', 'shoulder', 'season']
Оригинал: S&P Global PointLogic data points to temperatures within 2 

In [15]:
# Основная функция поиска
def find_near_words(result, processed_list, find):
    lemmatized_find = process_text(find)  # Лемматизируем и очищаем find через process_text

    found_indices = []  # Список для хранения индексов найденных предложений

    for i, sentence_tokens in enumerate(result):  # Здесь sentence_tokens — это уже список слов
        if len(lemmatized_find) == 1:  # Если в find только одно слово
            # Проверяем наличие этого слова в предложении
            if lemmatized_find[0] in sentence_tokens:
                found_indices.append(i)  # Если найдено, добавляем индекс предложения
        else:
            # Ищем, находятся ли леммы из find на расстоянии не более двух слов друг от друга
            positions = []
            for lemma in lemmatized_find:
                if lemma in sentence_tokens:
                    positions.append(sentence_tokens.index(lemma))

            # Если леммы найдены и их позиции расположены в пределах двух слов друг от друга
            if len(positions) == len(lemmatized_find):  # Все леммы найдены
                positions.sort()  # Сортируем позиции
                if all(abs(positions[i] - positions[i+1]) <= 2 for i in range(len(positions) - 1)):
                    found_indices.append(i)  # Добавляем индекс предложения

    # Выводим предложения из processed_list с найденными индексами
    for index in found_indices:
        print(f"Matching sentence in processed_list: {processed_list[index]}")


find = "global energy"

find_near_words(result, processed_list, find)

Matching sentence in processed_list: The report assumes that global final energy consumption will have dropped by 2% by 2050 and that electricity production willCoal Trader 
Matching sentence in processed_list: Stay current on the most significant events in global energy and commodities markets and connected with the industry’s most influential thought leaders. Visit cilive.com
